In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

In [2]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [3]:
load_dotenv()
chat= ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'),model="gpt-3.5-turbo")

In [4]:
df_temp= pd.read_csv("https://datahub.io/core/global-temp/r/0.csv")
df_temp.head()

,Source,Year,Mean
0,gcag,1850,-0.4177
1,gcag,1851,-0.2333
2,gcag,1852,-0.2294
3,gcag,1853,-0.2704
4,gcag,1854,-0.2915


In [6]:
agent_temp= create_pandas_dataframe_agent(
    chat,
    df_temp,
    verbose=True,
    agent_type="tool-calling"
)


In [8]:
# qual a temperatura media global do ultimo ano
resposta= agent_temp.invoke({"input":"Qual a temperatura média global do último ano?"})
print(resposta['output'])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Year'] == df['Year'].max()]['Mean'].values[0]"}`


1.1755A temperatura média global do último ano é de 1.1755.

> Finished chain.
A temperatura média global do último ano é de 1.1755.


In [9]:

resposta =agent_temp.invoke({"input":"Qual o ano com a maior temperatura média?"})
print(resposta['output'])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Mean'] == df['Mean'].max()]['Year'].values[0]"}`


2024O ano com a maior temperatura média é 2024.

> Finished chain.
O ano com a maior temperatura média é 2024.


TRABALHANDO COM AGENTES E BANCO DE DADOS SAQUILA DB

In [33]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent


In [39]:
db_sakila= SQLDatabase.from_uri("sqlite:///files/sqlite-sakila.db")

In [40]:
agent_executor_sakila=create_sql_agent(
    chat,
    db=db_sakila,
    agent_type="tool-calling",
    verbose=True
)

In [41]:
resposta = agent_executor_sakila.invoke({"input":"Quais são os filmes mais alugados?"})
print(resposta['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'film'}`



CREATE TABLE film (
	film_id INTEGER NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT DEFAULT NULL, 
	release_year VARCHAR(4) DEFAULT NULL, 
	language_id SMALLINT NOT NULL, 
	original_language_id SMALLINT DEFAULT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate DECIMAL(4, 2) DEFAULT (4.99) NOT NULL, 
	length SMALLINT DEFAULT NULL, 
	replacement_cost DECIMAL(5, 2) DEFAULT (19.99) NOT NULL, 
	rating VARCHAR(10) DEFAULT 'G', 
	special_features VARCHAR(100) DEFAULT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (film_id), 
	CONSTRAINT fk_film_language FOREIGN KEY(language_id) REFERENCES language (language_id), 
	CONSTRAINT fk_film_language_original FOREIGN KEY(orig

/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:313: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:325: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._met


Invoking: `sql_db_query` with `{'query': 'SELECT title, rental_duration, rental_rate, replacement_cost FROM film ORDER BY rental_rate DESC LIMIT 10'}`


[('ACE GOLDFINGER', 3, 4.99, 12.99), ('AIRPLANE SIERRA', 6, 4.99, 28.99), ('AIRPORT POLLOCK', 6, 4.99, 15.99), ('ALADDIN CALENDAR', 6, 4.99, 24.99), ('ALI FOREVER', 4, 4.99, 21.99), ('AMELIE HELLFIGHTERS', 4, 4.99, 23.99), ('AMERICAN CIRCUS', 3, 4.99, 17.99), ('ANTHEM LUKE', 5, 4.99, 16.99), ('APACHE DIVINE', 5, 4.99, 16.99), ('APOCALYPSE FLAMINGOS', 6, 4.99, 11.99)]Os filmes mais alugados são:

1. Título: ACE GOLDFINGER, Duração do aluguel: 3 dias, Taxa de aluguel: $4.99, Custo de reposição: $12.99
2. Título: AIRPLANE SIERRA, Duração do aluguel: 6 dias, Taxa de aluguel: $4.99, Custo de reposição: $28.99
3. Título: AIRPORT POLLOCK, Duração do aluguel: 6 dias, Taxa de aluguel: $4.99, Custo de reposição: $15.99
4. Título: ALADDIN CALENDAR, Duração do aluguel: 6 dias, Taxa de aluguel: $4.99, Custo de reposição: $24.99
5. Título: ALI FORE

In [42]:
resposta = agent_executor_sakila.invoke({"input":"Quais atores atuaram em mais filmes?"})
print(resposta['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'actor, film_actor'}`



CREATE TABLE actor (
	actor_id NUMERIC NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id)
)

/*
3 rows from actor table:
actor_id	first_name	last_name	last_update
1.0000000000	PENELOPE	GUINESS	2021-03-06 15:51:59
2.0000000000	NICK	WAHLBERG	2021-03-06 15:51:59
3.0000000000	ED	CHASE	2021-03-06 15:51:59
*/


CREATE TABLE film_actor (
	actor_id INTEGER NOT NULL, 
	film_id INTEGER NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id, film_id), 
	CONSTRAINT fk_film_actor_actor FOREIGN KEY(actor_id) REFERENCES actor (actor_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_film_actor_film FOREIGN KEY(f

/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:313: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:325: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._met


Invoking: `sql_db_query` with `{'query': 'SELECT actor_id, first_name, last_name, COUNT(film_id) AS film_count FROM actor JOIN film_actor ON actor.actor_id = film_actor.actor_id GROUP BY actor_id ORDER BY film_count DESC LIMIT 10'}`


Error: (sqlite3.OperationalError) ambiguous column name: actor_id
[SQL: SELECT actor_id, first_name, last_name, COUNT(film_id) AS film_count FROM actor JOIN film_actor ON actor.actor_id = film_actor.actor_id GROUP BY actor_id ORDER BY film_count DESC LIMIT 10]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_query` with `{'query': 'SELECT actor.actor_id, actor.first_name, actor.last_name, COUNT(film_actor.film_id) AS film_count FROM actor JOIN film_actor ON actor.actor_id = film_actor.actor_id GROUP BY actor.actor_id ORDER BY film_count DESC LIMIT 10'}`


[(107, 'GINA', 'DEGENERES', 42), (102, 'WALTER', 'TORN', 41), (198, 'MARY', 'KEITEL', 40), (181, 'MATTHEW', 'CARREY', 39), (23, 'SANDRA', 'KILMER', 37), (81, 'SCARLETT', 'D

In [43]:
resposta = agent_executor_sakila.invoke({"input":"Quais são os clientes que alugaram mais filmes?"})
print(resposta['output'])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'customer, rental'}`



CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	store_id INTEGER NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	email VARCHAR(50) DEFAULT NULL, 
	address_id INTEGER NOT NULL, 
	active CHAR(1) DEFAULT 'Y' NOT NULL, 
	create_date TIMESTAMP NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (customer_id), 
	CONSTRAINT fk_customer_store FOREIGN KEY(store_id) REFERENCES store (store_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_customer_address FOREIGN KEY(address_id) REFERENCES address (address_id) ON UPDATE CASCADE
)

/*
3 rows from customer table:
customer_id	store_id	first_name	last_name	email	address_id	active	create_date	last_update
1	1	MARY

/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:313: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/home/joelct/Area_de_Trabalho/Curso Engenharia IA/analis_sentiment/.venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:325: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._met


Invoking: `sql_db_query` with `{'query': 'SELECT c.first_name, c.last_name, COUNT(r.customer_id) AS num_rentals FROM customer c JOIN rental r ON c.customer_id = r.customer_id GROUP BY c.customer_id ORDER BY num_rentals DESC LIMIT 10'}`


[('ELEANOR', 'HUNT', 46), ('KARL', 'SEAL', 45), ('CLARA', 'SHAW', 42), ('MARCIA', 'DEAN', 42), ('TAMMY', 'SANDERS', 41), ('SUE', 'PETERS', 40), ('WESLEY', 'BULL', 40), ('RHONDA', 'KENNEDY', 39), ('MARION', 'SNYDER', 39), ('TIM', 'CARY', 39)]Os clientes que alugaram mais filmes são:

1. Eleanor Hunt - 46 aluguéis
2. Karl Seal - 45 aluguéis
3. Clara Shaw - 42 aluguéis
4. Marcia Dean - 42 aluguéis
5. Tammy Sanders - 41 aluguéis
6. Sue Peters - 40 aluguéis
7. Wesley Bull - 40 aluguéis
8. Rhonda Kennedy - 39 aluguéis
9. Marion Snyder - 39 aluguéis
10. Tim Cary - 39 aluguéis

> Finished chain.
Os clientes que alugaram mais filmes são:

1. Eleanor Hunt - 46 aluguéis
2. Karl Seal - 45 aluguéis
3. Clara Shaw - 42 aluguéis
4. Marcia Dean - 42 aluguéis
5. Tammy 